<a href="https://colab.research.google.com/github/StevenZhang904/Final-Project/blob/master/ECON3382datacleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
import numpy as np
import pandas as pd
import requests
import json

In [20]:

blockNum = 100000
r = requests.get( 'https://blockchain.info/block-height/'+str(blockNum)+'?format-json')
block = r.json()
print (block)

{'blocks': [{'hash': '000000000003ba27aa200b1cecaad478d2b00432346c3f1f3986da1afd33e506', 'ver': 1, 'prev_block': '000000000002d01c1fccc21636b607dfd930d31d01c3a62104612a1719011250', 'mrkl_root': 'f3e94742aca4b5ef85488dc37c06c3282295ffec960994b2c0d5ac2a25a95766', 'time': 1293623863, 'bits': 453281356, 'next_block': ['00000000000080b66c911bd5ba14a74260057311eaeb1982802f7010f1a9f090'], 'fee': 0, 'nonce': 274148111, 'n_tx': 4, 'size': 957, 'block_index': 100000, 'main_chain': True, 'height': 100000, 'weight': 3828, 'tx': [{'hash': '8c14f0db3df150123e6f3dbbf30f8b955a8249b62ac1d1ff16284aefa3d06d87', 'ver': 1, 'vin_sz': 1, 'vout_sz': 1, 'size': 135, 'weight': 540, 'fee': 0, 'relayed_by': '0.0.0.0', 'lock_time': 0, 'tx_index': 4764983090866501, 'double_spend': False, 'time': 1293623863, 'block_index': 100000, 'block_height': 100000, 'inputs': [{'sequence': 4294967295, 'witness': '', 'script': '044c86041b020602', 'index': 0, 'prev_out': None}], 'out': [{'type': 0, 'spent': True, 'value': 5000000

In [32]:
dataDF = pd.json_normalize(block, record_path= ["blocks"])
txs = pd.json_normalize(dataDF["tx"][0])
txs
#\inputAddresses = pd.json_normalize(txs)
#inputAddresses

,hash,ver,vin_sz,vout_sz,size,weight,fee,relayed_by,lock_time,tx_index,double_spend,time,block_index,block_height,inputs,out
0,8c14f0db3df150123e6f3dbbf30f8b955a8249b62ac1d1...,1,1,1,135,540,0,0.0.0.0,0,4764983090866501,False,1293623863,100000,100000,"[{'sequence': 4294967295, 'witness': '', 'scri...","[{'type': 0, 'spent': True, 'value': 500000000..."
1,fff2525b8931402dd09222c50775608f75787bd2b87e56...,1,1,2,259,1036,0,0.0.0.0,0,6896493393222575,False,1293623863,100000,100000,"[{'sequence': 4294967295, 'witness': '', 'scri...","[{'type': 0, 'spent': True, 'value': 556000000..."
2,6359f0868171b1d194cbee1af2f16ea598ae8fad666d9b...,1,1,2,257,1028,0,0.0.0.0,0,6911274329242193,False,1293623863,100000,100000,"[{'sequence': 4294967295, 'witness': '', 'scri...","[{'type': 0, 'spent': True, 'value': 1000000, ..."
3,e9a66845e05d5abc0ad04ec80f774a7e585c6e8db97596...,1,1,1,225,900,0,0.0.0.0,0,1022094957881939,False,1293623863,100000,100000,"[{'sequence': 4294967295, 'witness': '', 'scri...","[{'type': 0, 'spent': True, 'value': 1000000, ..."


In [56]:
dataDF

,hash,ver,prev_block,mrkl_root,time,bits,next_block,fee,nonce,n_tx,size,block_index,main_chain,height,weight,tx
0,000000000003ba27aa200b1cecaad478d2b00432346c3f...,1,000000000002d01c1fccc21636b607dfd930d31d01c3a6...,f3e94742aca4b5ef85488dc37c06c3282295ffec960994...,1293623863,453281356,[00000000000080b66c911bd5ba14a74260057311eaeb1...,0,274148111,4,957,100000,True,100000,3828,[{'hash': '8c14f0db3df150123e6f3dbbf30f8b955a8...


In [85]:
a= pd.DataFrame(txs,columns=['inputs','out'])
a["inputs"][]


0    [{'sequence': 4294967295, 'witness': '', 'scri...
1    [{'sequence': 4294967295, 'witness': '', 'scri...
2    [{'sequence': 4294967295, 'witness': '', 'scri...
3    [{'sequence': 4294967295, 'witness': '', 'scri...
Name: inputs, dtype: object

In [132]:
b = pd.json_normalize(a["out"][1])
b

,type,spent,value,spending_outpoints,n,tx_index,script,addr
0,0,True,556000000,"[{'tx_index': 316661668806251, 'n': 6}]",0,6896493393222575,76a914c398efa9c392ba6013c5e04ee729755ef7f58b32...,1JqDybm2nWTENrHvMyafbSXXtTk5Uv5QAn
1,0,True,4444000000,"[{'tx_index': 4322279565170935, 'n': 0}]",1,6896493393222575,76a914948c765a6914d43f2a7ac177da2c2f6b52de3d7c...,1EYTGtG4LnFfiMvjJdsU7GMGCQvsRSjYhx


In [63]:
def getBlock(blockNum):
  r = requests.get( 'https://blockchain.info/block-height/'+str(blockNum)+'?format-json')
  block = r.json()
  return block

In [133]:
def parse(block):
    inputAddresses = []
    outputAddresses = []
    inputValue = []
    outputValue = []
    timeStamp = []
    blockNumber = []
    dataDF = pd.json_normalize(block, record_path= ["blocks"])
    txs = pd.json_normalize(dataDF["tx"][0])
    parsedTxs = pd.DataFrame(txs,columns=['inputs','out'])
    #loop through the transactions with inputs and outputs individually
    for x in range(1,len(parsedTxs["inputs"])): 
        Input = pd.json_normalize(parsedTxs["inputs"][x])
        Output = pd.json_normalize(parsedTxs["out"][x])
        inputAddresses.append(Input["prev_out.addr"])
        inputValue.append(Input["prev_out.value"])
        outputAddresses.append(Output["addr"])
        outputValue.append(Output["value"])
        timeStamp.append(dataDF["time"][0])
        blockNumber.append(dataDF["height"][0])
    data = np.array([inputAddresses, 
                     outputAddresses, 
                     inputValue,
                     outputValue, 
                     timeStamp, 
                     blockNumber])
    #transpose the dataframe, switching the rows and columns
    result = data.T
    df = pd.DataFrame(result)
    df.columns = ["inputAddresses", 
                  "outputAddresses", 
                  "inputValue", 
                  "outputValue", 
                  "timeStamp", 
                  "blockNumber"]
    return df


In [135]:
test = getBlock(100000)
df = parse(test)
df.to_csv("/ECON3382/test.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
